In [1]:
import ee
import geemap

ee.Initialize()

In [2]:
def get_s1_collection(aoi: ee.Geometry, start_date: str, end_date: str, lookdir: str = None) -> ee.ImageCollection:
    """Get Sentinel-1 GRD data for a given area and time period.

    Args:
        aoi (ee.Geometry): Area of interest.
        start_date (str): Start date in YYYY-MM-DD format.
        end_date (str): End date in YYYY-MM-DD format.

    Returns:
        ee.ImageCollection: Sentinel-1 GRD data.
    """
    if lookdir not in ['ASCENDING', 'DESCENDING']:
        raise ValueError('lookdir must be either ASCENDING or DESCENDING')
    
    s1 = (
        ee.ImageCollection('COPERNICUS/S1_GRD')
        .filterBounds(aoi)
        .filterDate(start_date, end_date)
        .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
        .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))
        .filter(ee.Filter.eq('instrumentMode', 'IW'))
        .filter(ee.Filter.eq('orbitProperties_pass', lookdir))
        .filter(ee.Filter.eq('resolution_meters', 10))
        .select(['VV', 'VH'])
    )
    return s1

In [3]:
def compute():
    for year in range(2017, 2022):
        aoi = ee.FeatureCollection("projects/yk-wetland-mapping/assets/yukon-bounds")
        start_mm_dd = f'{year}-06-20'
        end_mm_dd = f'{year}-09-22'
        asc = get_s1_collection(aoi, start_mm_dd, end_mm_dd, lookdir='ASCENDING')
        desc = get_s1_collection(aoi, start_mm_dd, end_mm_dd, lookdir='DESCENDING')
        yield asc.median(), desc.median(), str(year)

s1_collections = compute()


In [9]:
Map = geemap.Map()
vis = {'bands': ['VV'],'min': -25, 'max': 10}
asc, desc, year = next(s1_collections)
Map.addLayer(asc, vis, f'S1 ASC: {year}')
Map.addLayer(desc, vis, f'S1 DESC: {year}')
Map.addLayerControl()

Map

StopIteration: 